In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn import svm
import numpy as np
import argparse
import imutils  # If you are unable to install this library, ask the TA; we only need this in extract_hsv_histogram.
import cv2  # cv2 
import os
import random
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import data, exposure
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score






Preprocess the image 

1- aplly gaussian blur to remove noise


2- apply median blur to remove salt and pepper noise


3- resize image


4- mormalize pixels values

In [3]:
def preprocess_image_fromPath(image_path, target_size=(64, 64)):
    """
    Preprocess the image by resizing, grayscaling, applying noise reduction, and normalizing.
    Args:
        image_path: Path to the image.
        target_size: Target size for resizing.
    Returns:
        Preprocessed image.
    """
    
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found at {image_path}")
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply a Median filter for further noise reduction (optional)
    filtered = cv2.medianBlur(blurred, 3)
    
    # Resize to the target size
    resized = cv2.resize(filtered, target_size)
    
    # Normalize pixel values to [0, 1]
    normalized = resized / 255.0
    
    return normalized

def preprocess_image(image, target_size):
    """
    Preprocess the image by resizing, grayscaling, applying noise reduction, and normalizing.
    Args:
        image: Input image.
        target_size: Target size for resizing.
    Returns:
        Preprocessed image.
    """
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply a Median filter for further noise reduction (optional)
    filtered = cv2.medianBlur(blurred, 3)
    
    # Resize to the target size
    resized = cv2.resize(filtered, target_size)
    
    # Normalize pixel values to [0, 1]
    normalized = resized / 255.0
    
    return normalized


Extract HOG features 


def extract_hog_features(...): This function is defined to compute Histogram of Oriented Gradients (HOG) features from a given grayscale image. HOG features describe the distribution of gradient orientations in an image, useful for object detection and recognition.


image: Input parameter for the preprocessed image, expected to be in grayscale format.


orientations: Number of orientation bins used to compute the gradient histogram. Default is 9, meaning the gradient direction is divided into 9 bins (e.g., 0°–20°, 20°–40°, etc.).


pixels_per_cell: Defines the size of each cell in pixels. The gradient histogram is computed for each cell. Default size is 8x8 pixels.


cells_per_block: Specifies how many cells are grouped into a block for normalization. Default is a 2x2 block of cells.


In [5]:

def extract_hog_features(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    """
    Extract HOG features from the image.
    Args:
        image: Preprocessed image (grayscale).
        orientations: Number of orientation bins.
        pixels_per_cell: Size of the cell in pixels.
        cells_per_block: Number of cells in each block.
    Returns:
        HOG feature vector.
    """


    hog_features = hog(image, orientations=orientations,
                       pixels_per_cell=pixels_per_cell,
                       cells_per_block=cells_per_block,
                       block_norm='L2-Hys', visualize=False)
    '''
hog(image, ...): This function, imported from skimage.feature, computes the HOG features for the input image based on the specified parameters.

orientations=orientations: Specifies the number of bins for the gradient histogram.

pixels_per_cell=pixels_per_cell: Defines the dimensions of the cells for which the gradient histogram is computed.

cells_per_block=cells_per_block: Specifies the size of the block used for local contrast normalization.

block_norm='L2-Hys': Indicates the normalization method to use for the blocks. L2-Hys is a widely used normalization technique that improves robustness to lighting and contrast changes.

visualize=False: Disables the visualization of the HOG image, which is useful for debugging but not required for feature extraction.
'''
    return hog_features


Build list of features and labels for training the model 

In [74]:

# def load_dataset(dataset_path, target_size=(64, 64)):
#     """
#     Load images, preprocess, and extract HOG features.
#     Args:
#         dataset_path: Root directory of the dataset.
#         target_size: Target size for resizing.
#     Returns:
#         Features, labels, and class mapping.
#     """
#     features = []
#     labels = []
#     class_mapping = {}

#     # Generate class mapping dynamically
#     class_names = sorted(os.listdir(dataset_path))  # Sort for consistent label assignment
#     for idx, class_name in enumerate(class_names):
#         class_mapping[class_name] = idx  # Assign an index to each class

#     for class_name, label in class_mapping.items():
#         class_path = os.path.join(dataset_path, class_name)
#         for file_name in os.listdir(class_path):
#             image_path = os.path.join(class_path, file_name)
#             # Preprocess the image
#             preprocessed_image = preprocess_image_fromPath(image_path, target_size)
#             # Extract HOG features
#             feature_vector = extract_hog_features(preprocessed_image)
#             features.append(feature_vector)
#             labels.append(label)

#     return np.array(features), np.array(labels), class_mapping

# # Define dataset path
# dataset_path = r"D:\CUFE\GP\Sleep_detection\archive\mrleyedataset"

# # Load dataset
# X, y, class_mapping = load_dataset(dataset_path)

# # Split into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Print the class mapping
# print("Class Mapping:", class_mapping)


In [7]:
import pickle


# pickle_out = open("class_mapping.pickle","wb")
# pickle.dump(class_mapping, pickle_out)
# pickle_out.close()

# pickle_out = open("X_train.pickle","wb")
# pickle.dump(X_train, pickle_out)
# pickle_out.close()

# pickle_out = open("X_test.pickle","wb")
# pickle.dump(X_test, pickle_out)
# pickle_out.close()

# pickle_out = open("y_train.pickle","wb")
# pickle.dump(y_train, pickle_out)
# pickle_out.close()

# pickle_out = open("y_test.pickle","wb")
# pickle.dump(y_test, pickle_out)
# pickle_out.close()

pickle_in = open("class_mapping.pickle","rb")
class_mapping = pickle.load(pickle_in)

# pickle_in = open("X_train.pickle","rb")
# X_train = pickle.load(pickle_in)

# pickle_in = open("X_test.pickle","rb")
# X_test = pickle.load(pickle_in)

# pickle_in = open("y_train.pickle","rb")
# y_train = pickle.load(pickle_in)

# pickle_in = open("y_test.pickle","rb")
# y_test = pickle.load(pickle_in)



1- Train the model

2- Test the model

3- Evaluate the model

In [76]:
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score

# # Train the SVM
# svm_model = SVC(kernel='linear', random_state=42)
# svm_model.fit(X_train, y_train)

# # Evaluate the SVM
# y_pred = svm_model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"SVM Accuracy: {accuracy * 100:.2f}%")


In [10]:
# save the model to disk
filename = 'SVM_model.sav'
# pickle.dump(svm_model, open(filename, 'wb'))

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, y_test)
# print(result)


Test the model on test images

In [11]:
def predict_image_fromPath(image_path, model, class_mapping, target_size=(64, 64)):
    """
    Predict the class of a new image.
    Args:
        image_path: Path to the image.
        model: Trained SVM model.
        class_mapping: Mapping of class names to labels.
        target_size: Target size for resizing.
    Returns:
        Predicted class name.
    """
    # Preprocess and extract features
    preprocessed_image = preprocess_image_fromPath(image_path, target_size)
    feature_vector = extract_hog_features(preprocessed_image).reshape(1, -1)
    # Predict
    prediction = model.predict(feature_vector)[0]
    # Map label back to class name
    for class_name, label in class_mapping.items():
        if label == prediction:
            return class_name

def predict_image(image, model, class_mapping, target_size=(224, 224)):
    """
    Predict the class of a new image.
    Args:
        image: Input image.
        model: Trained model.
        class_mapping: Dictionary mapping class indices to class names.
        target_size: Target size for resizing.
    Returns:
        Predicted class name.
    """
    # Preprocess and extract features
    preprocessed_image = preprocess_image(image, target_size)
    feature_vector = extract_hog_features(preprocessed_image).reshape(1, -1)
    # Predict
    prediction = model.predict(feature_vector)[0]
    # Map label back to class name
    for class_name, label in class_mapping.items():
        if label == prediction:
            return class_name

Test

In [79]:
# prdicting the image
image_path = r"D:\CUFE\GP\Sleep_detection\archive\open_eye.jpg"
predicted_class = predict_image_fromPath(image_path, loaded_model, class_mapping)
print(f"Predicted Class: {predicted_class}")

Predicted Class: Open-Eyes


In [21]:
img = cv2.imread("archive/man_open.png")
# plt.imshow(img)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)
i = 0
for (x, y, w, h) in eyes:
    i += 1
    print(f"Eye {i}: ({x}, {y}, {w}, {h})")
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)  
    
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))




NameError: name 'cv2' is not defined

In [20]:
#CROP THE EYES
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)
# eyes_roi = []
for x, y, w, h in eyes:
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyess = eye_cascade.detectMultiScale(roi_gray)
    if len(eyess) == 0:
        print("eyes are not detected")
    else:
        for (ex, ey, ew, eh) in eyess:
            eyes_roi = roi_color[ey: ey+eh, ex: ex+ew]

plt.imshow(cv2.cvtColor(eyes_roi, cv2.COLOR_BGR2RGB))

NameError: name 'cv2' is not defined

In [19]:
eyes_roi.shape

NameError: name 'eyes_roi' is not defined

In [83]:
# save the image
cv2.imwrite("archive/test.jpg", eyes_roi)

True

In [84]:
test = predict_image_fromPath("archive/test.jpg", loaded_model, class_mapping)
print(f"Predicted Class: {test}")

Predicted Class: Open-Eyes


In [ ]:
# import cv2
# import time
# import winsound  # For beep sound (Windows)


# # Load Haar cascade classifiers
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# cap = cv2.VideoCapture(0)
# if not cap.isOpened():
#     cap = cv2.VideoCapture(1)
# if not cap.isOpened():
#     raise IOError("Cannot open webcam")

# eyes_closed_time = None  # Track when eyes were first closed
# ALERT_THRESHOLD = 3  # Seconds before triggering an alert

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = face_cascade.detectMultiScale(gray, 1.1, 4)

#     eyes_detected = False  # Default: No eyes detected

#     for (x, y, w, h) in faces:
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
#         roi_gray = gray[y:y + h, x:x + w]
#         roi_color = frame[y:y + h, x:x + w]

#         eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 4)

#         if len(eyes) > 0:
#             eyes_detected = True  # Eyes detected, reset timer
#             for (ex, ey, ew, eh) in eyes:
#                 cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (255, 0,
#                                                                             0), 2)
#                 #use model to predict the eye
#                 eyes_roi = roi_color[ey: ey+eh, ex: ex+ew]
#                 #USING THE MODEL TO PREDICT THE EYE
#                 test = predict_image(eyes_roi, loaded_model, class_mapping)
#                 print(f"Predicted Class: {test}")
    


#     # Check if eyes are closed or not detected
#     if not eyes_detected:
#         if eyes_closed_time is None:  # Start timing when eyes close/disappear
#             eyes_closed_time = time.time()
#         elif time.time() - eyes_closed_time >= ALERT_THRESHOLD:
#             print("ALERT! Eyes closed/not detected for 3 seconds!")
#             winsound.Beep(2500, 1000)  # Beep sound (2500 Hz, 1 sec)
#             eyes_closed_time = None  # Reset timer after alert
#     else:
#         eyes_closed_time = None  # Reset if eyes open

#     cv2.imshow('Drowsiness Detection', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


ValueError: X has 26244 features, but SVC is expecting 1764 features as input.

: 

In [ ]:
import cv2
import numpy as np
import time
import os
import pickle

# Ensure archive directory exists
if not os.path.exists("archive"):
    os.makedirs("archive")

# Load the model
loaded_model = pickle.load(open("SVM_model.sav", 'rb'))
class_mapping = pickle.load(open("class_mapping.pickle", "rb"))

# Eye Tracker with improved glasses handling
class ImprovedEyeTracker:
    def __init__(self):
        # State tracking
        self.state = "Unknown"
        self.confidence = 0.0
        self.closed_time_start = None
        self.alert_active = False
        self.alert_threshold = 3.0  # Seconds before alerting
        
        # Enhanced history management
        self.history = []
        self.history_size = 30  # Longer history for stability
        self.state_stability_threshold = 0.65  # Higher threshold for state changes
        
        # Glasses-specific settings
        self.has_glasses = False
        self.glasses_detection_count = 0
        self.glasses_threshold = 5  # Need this many consecutive detections to confirm glasses
        
        # Detection info
        self.detection_method = None
        self.last_successful_detection = time.time()
        self.detection_timeout = 1.0  # Seconds before considering detection lost
        
        # Last known good regions
        self.last_eye_roi = None
        
    def update(self, eye_state, detection_confidence, detection_method, has_glasses_frame=False):
        current_time = time.time()
        
        # Update glasses detection
        if has_glasses_frame:
            self.glasses_detection_count += 1
        else:
            self.glasses_detection_count = max(0, self.glasses_detection_count - 1)
        
        # Determine if wearing glasses (needs multiple consistent detections)
        self.has_glasses = self.glasses_detection_count >= self.glasses_threshold
        
        # Update detection method
        self.detection_method = detection_method
        
        # Add to history for smoothing
        self.history.append(eye_state)
        if len(self.history) > self.history_size:
            self.history.pop(0)
        
        # Count state occurrences
        state_counts = {}
        for state in self.history:
            if state in state_counts:
                state_counts[state] += 1
            else:
                state_counts[state] = 1
        
        # Get most common state with sufficient history
        if len(self.history) >= 5:  # Need at least 5 detections to make a decision
            most_common = max(state_counts.items(), key=lambda x: x[1])
            smoothed_state = most_common[0]
            self.confidence = most_common[1] / len(self.history)
            
            # Update state if confident enough
            if self.confidence > self.state_stability_threshold:
                if self.state != smoothed_state:
                    print(f"State changed: {smoothed_state} (via {detection_method}, confidence: {self.confidence:.2f})")
                    self.state = smoothed_state
        
        # Track closed eyes for drowsiness detection
        if self.state == "Close-Eyes":
            if self.closed_time_start is None:
                self.closed_time_start = current_time
            # Do nothing else - just keep the timer running
        else:
            self.closed_time_start = None
            self.alert_active = False

    def check_drowsiness(self):
        if self.closed_time_start is not None:
            elapsed = time.time() - self.closed_time_start
            if elapsed >= self.alert_threshold and not self.alert_active:
                self.alert_active = True
                return True
        return False
    
    def get_closed_duration(self):
        if self.closed_time_start is None:
            return 0
        return time.time() - self.closed_time_start

# Specialized preprocessing for different lighting conditions
def enhance_for_glasses(image):
    if image is None or image.size == 0:
        return None
        
    # Convert to grayscale if needed
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
    
    # Apply advanced preprocessing specifically for glasses
    # 1. CLAHE for better contrast around eyes with glasses
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)
    
    # 2. Bilateral filter to preserve edges while reducing noise
    enhanced = cv2.bilateralFilter(enhanced, 9, 25, 25)
    
    # 3. Gamma correction to better handle reflections on glasses
    gamma = 1.2
    lookUpTable = np.empty((1,256), np.uint8)
    for i in range(256):
        lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)
    enhanced = cv2.LUT(enhanced, lookUpTable)
    
    return enhanced

# Main application
def run_eye_detection():
    # Initialize cascades with optimized parameters for glasses
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    eyeglasses_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye_tree_eyeglasses.xml')
    
    # Verify cascades loaded properly
    if face_cascade.empty() or eye_cascade.empty() or eyeglasses_cascade.empty():
        print("Error: Couldn't load cascade classifiers!")
        return
    
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
    
    # Create eye tracker
    tracker = ImprovedEyeTracker()
    
    # Main loop
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Create a copy for display
        display = frame.copy()
        
        # Basic grayscale conversion
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Apply CLAHE for better contrast globally
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        enhanced_gray = clahe.apply(gray)
        
        # State variables for this frame
        eye_state = "Unknown"
        detection_confidence = 0.0
        detection_method = "None"
        has_glasses_detected = False
        
        # Detect faces first
        faces = face_cascade.detectMultiScale(
            enhanced_gray, 
            scaleFactor=1.1, 
            minNeighbors=5, 
            minSize=(80, 80),
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        
        if len(faces) > 0:
            # Process the largest face
            x, y, w, h = max(faces, key=lambda face: face[2] * face[3])
            
            # Draw face rectangle
            cv2.rectangle(display, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Extract face region with some margin
            margin = int(h * 0.1)  # 10% margin
            face_y1 = max(0, y-margin)
            face_y2 = min(frame.shape[0], y+h+margin)
            face_x1 = max(0, x-margin)
            face_x2 = min(frame.shape[1], x+w+margin)
            
            face_gray = enhanced_gray[face_y1:face_y2, face_x1:face_x2]
            face_color = frame[face_y1:face_y2, face_x1:face_x2]
            
            # Specially enhanced version for glasses detection
            glasses_enhanced = enhance_for_glasses(face_gray)
            
            # First try glasses-specific cascade with relaxed parameters
            eyes_with_glasses = eyeglasses_cascade.detectMultiScale(
                glasses_enhanced, 
                scaleFactor=1.05,  # Smaller scale factor for better detection
                minNeighbors=3,    # Fewer neighbors required for glasses
                minSize=(25, 25),  # Smaller minimum size
                flags=cv2.CASCADE_SCALE_IMAGE
            )
            
            if len(eyes_with_glasses) > 0:
                # Glasses detected
                eyes = eyes_with_glasses
                has_glasses_detected = True
                detection_method = "Glasses-Cascade"
                
                # Draw all detected glasses eyes
                for (ex, ey, ew, eh) in eyes:
                    cv2.rectangle(face_color, (ex, ey), (ex+ew, ey+eh), (255, 0, 0), 2)
                
                # Use the most likely to be an eye (typically higher in the face)
                # Sort by y-coordinate (smaller = higher in face)
                eyes_sorted = sorted(eyes, key=lambda e: e[1])
                
                # Take the top eye if we found multiple
                ex, ey, ew, eh = eyes_sorted[0] if len(eyes_sorted) > 0 else eyes[0]
                
            else:
                # Try regular eye detector with optimized parameters
                eyes = eye_cascade.detectMultiScale(
                    face_gray, 
                    scaleFactor=1.05, 
                    minNeighbors=6,     # More neighbors for better accuracy
                    minSize=(25, 25),
                    flags=cv2.CASCADE_SCALE_IMAGE
                )
                
                has_glasses_detected = False
                detection_method = "Regular-Cascade"
                
                if len(eyes) > 0:
                    # Draw detected eyes
                    for (ex, ey, ew, eh) in eyes:
                        cv2.rectangle(face_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 255), 2)
                    
                    # If multiple eyes detected, use the one closer to expected position
                    if len(eyes) >= 2:
                        # Sort by y-coordinate to get upper eyes
                        eyes_sorted = sorted(eyes, key=lambda e: e[1])
                        # Take the first eye
                        ex, ey, ew, eh = eyes_sorted[0]
                    else:
                        ex, ey, ew, eh = eyes[0]
            
            # If eyes were detected with either method
            if len(eyes) > 0:
                # Extract the eye region
                eye_roi = face_color[ey:ey+eh, ex:ex+ew]
                
                # Save the eye ROI for later use
                if eye_roi.size > 0:
                    tracker.last_eye_roi = eye_roi
                    
                    # Save and predict
                    cv2.imwrite("archive/test.jpg", eye_roi)
                    try:
                        eye_state = predict_image_fromPath("archive/test.jpg", loaded_model, class_mapping)
                        detection_confidence = 0.7
                        
                        # Draw prediction on frame
                        label = f"{eye_state} ({detection_confidence:.2f})"
                        cv2.putText(face_color, label, (ex, ey-5), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                    except Exception as e:
                        print(f"Prediction error: {e}")
                        eye_state = "Error"
                        detection_confidence = 0.3
            else:
                # No eyes detected in this frame
                eye_state = "Eyes Not Detected"
                detection_confidence = 0.4
                
                # If we have a saved eye ROI, try using that
                if tracker.last_eye_roi is not None:
                    try:
                        # Save the last known eye and classify
                        cv2.imwrite("archive/last_eye.jpg", tracker.last_eye_roi)
                        fallback_state = predict_image_fromPath("archive/last_eye.jpg", loaded_model, class_mapping)
                        
                        # Only use this if we're fairly sure eyes are still there but detection failed
                        if time.time() - tracker.last_successful_detection < tracker.detection_timeout:
                            eye_state = fallback_state
                            detection_confidence = 0.5
                            detection_method = "Last-ROI-Fallback"
                    except:
                        pass
        else:
            # No face detected
            eye_state = "No Face"
            detection_confidence = 0.2
            
        # Update tracker with detected eye state
        tracker.update(eye_state, detection_confidence, detection_method, has_glasses_detected)
        
        # If we detected eyes successfully, update the timestamp
        if eye_state not in ["No Face", "Eyes Not Detected", "Error", "Unknown"]:
            tracker.last_successful_detection = time.time()
        
        # Check for drowsiness
        if tracker.check_drowsiness():
            # Alert for drowsiness
            try:
                import winsound
                winsound.Beep(1000, 500)  # Beep at 1000 Hz for 500 ms
            except:
                print("ALERT: Drowsiness detected!")
        
        # Create info panel
        info_panel = np.zeros((200, frame.shape[1], 3), dtype=np.uint8)
        
        # Add information to the panel
        cv2.putText(info_panel, f"Eye State: {tracker.state}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        
        cv2.putText(info_panel, f"Detection: {tracker.detection_method}", (10, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        
        cv2.putText(info_panel, f"Confidence: {tracker.confidence:.2f}", (10, 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        
        # Use more visible indication for glasses detection
        glasses_color = (0, 255, 255) if tracker.has_glasses else (100, 100, 100)
        cv2.putText(info_panel, f"Glasses: {'YES' if tracker.has_glasses else 'NO'}", (10, 120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, glasses_color, 2)
        
        # Show drowsiness warning if necessary
        if tracker.closed_time_start is not None:
            elapsed = tracker.get_closed_duration()
            cv2.putText(info_panel, f"Eyes closed for: {elapsed:.1f}s", (10, 150), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 165, 255), 2)
            
            # Flash warning if alert is active
            if elapsed >= tracker.alert_threshold:
                if int(time.time() * 2) % 2 == 0:  # Flashing effect
                    cv2.putText(info_panel, "DROWSINESS ALERT!", (10, 190), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
        
        # Combine display and info panel
        combined = np.vstack((display, info_panel))
        
        # Show the result
        cv2.imshow("Drowsiness Detection", combined)
        
        # Exit on 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Start the application
run_eye_detection()

C:\Users\tarek\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator SVC from version 1.5.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


State changed: Open-Eyes (via Regular-Cascade, confidence: 1.00)
